# This is the example of KMC run for nasicon
By running the respective code block, the json file for kmc input should be generated. KMC will take the json as input paramter to execute the simulation

# 1. Local Cluster Expansion

## Required input 
- cif file of crystal structure: template_cif_fname
## Output
- lce.json
- lce_site.json

In [1]:
from kmcpy.model import LocalClusterExpansion

mobile_ion_identifier_type = "label"
mobile_ion_specie_1_identifier = "Na1"
lce = LocalClusterExpansion()
lce.initialization(
    mobile_ion_identifier_type=mobile_ion_identifier_type,
    mobile_ion_specie_1_identifier=mobile_ion_specie_1_identifier,
    cutoff_cluster=[8, 6, 0],
    cutoff_region=4,
    template_cif_fname="./file_before_kmc/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",
    convert_to_primitive_cell=True,
)
lce.to_json("lce.json")
lce.to_json("lce_site.json")

please check if these are mobile_ion_specie_1:
[ 0.      0.     11.0905] Na
[0. 0. 0.] Na


Centering the local environment at 0 ...
Searching local env around [ 0.      0.     11.0905] Na ...
793 clusters will be generated ...

Generating possible clusters within this migration unit...
Cutoffs: pair = 8 Angst, triplet = 6 Angst, quadruplet = 0 Angst
Type	Index	max_length	min_length	Point Group	Multiplicity
point	(np.int64(0),)	   0.000	   0.000	Kh   	    6
point	(np.int64(6),)	   0.000	   0.000	Kh   	    6
pair 	(np.int64(0), np.int64(1))	   4.727	   4.727	D*h  	    6
pair 	(np.int64(0), np.int64(3))	   5.103	   5.103	D*h  	    6
pair 	(np.int64(0), np.int64(5))	   6.956	   6.956	D*h  	    3
pair 	(np.int64(0), np.int64(6))	   3.710	   3.710	C*v  	    6
pair 	(np.int64(0), np.int64(7))	   3.056	   3.056	C*v  	    6
pair 	(np.int64(0), np.int64(8))	   3.162	   3.162	C*v  	    6
pair 	(np.int64(0), np.int64(9))	   6.492	   6.492	C*v  	    6
pair 	(np.int64(0), np.int64(10))	   6.543	   6.543	C*v  	    6
pair 	(np.int64(0), np.int64(11))	   6.196	   6.196	C*v  	    6
pair 	(np.

# 2. Fitting
Fit the Local Cluster Expansion model, data is from out KMC paper https://www.nature.com/articles/s41467-022-32190-7
## Required input 
- local cluster expansion correlation matrix, ekra, weight

    in the file\_before\_kmc/local\_cluster\_expansion
## Output
- fitting\_results.json

    fitting result of $E_{KRA}$ and $E_{site}$
- (keci.txt)

    not input file for KMC, just for checking the parameters


In [2]:
from kmcpy.fitting import Fitting

local_cluster_expansion_fit = Fitting()

y_pred, y_true = local_cluster_expansion_fit.fit(
    alpha=1.5,
    max_iter=1000000,
    ekra_fname="file_before_kmc/local_cluster_expansion/e_kra.txt",
    keci_fname="keci.txt",
    weight_fname="file_before_kmc/local_cluster_expansion/weight.txt",
    corr_fname="file_before_kmc/local_cluster_expansion/correlation_matrix.txt",
    fit_results_fname="fitting_results.json",
)
print("fitting", y_pred, y_true)
local_cluster_expansion_fit = Fitting()

y_pred, y_true = local_cluster_expansion_fit.fit(
    alpha=1.5,
    max_iter=1000000,
    ekra_fname="file_before_kmc/local_cluster_expansion_site/e_site.txt",
    keci_fname="keci.txt",
    weight_fname="file_before_kmc/local_cluster_expansion_site/weight.txt",
    corr_fname="file_before_kmc/local_cluster_expansion_site/correlation_matrix.txt",
    fit_results_fname="fitting_results_site.json",
)
print("fitting", y_pred, y_true)

Loading E_KRA from file_before_kmc/local_cluster_expansion/e_kra.txt ...
Lasso Results:
KECI = 
[ 18.08   0.     0.     0.    -0.49  -7.8    0.     0.   -21.     0.
  -2.72   0.    19.63 -17.54   0.     3.85   0.    -0.     9.16  -0.
   0.43  -0.     9.84  -6.77]
There are  12 Non Zero KECI
Empty Cluster = 
361.4390961351352
index	NEB	LCE	NEB-LCE
[[ 1.0000e+00  4.7325e+02  4.7059e+02  2.6600e+00]
 [ 2.0000e+00  4.4575e+02  4.7055e+02 -2.4800e+01]
 [ 3.0000e+00  4.3758e+02  4.2864e+02  8.9300e+00]
 [ 4.0000e+00  3.9735e+02  3.8858e+02  8.7700e+00]
 [ 5.0000e+00  4.4500e+02  4.4163e+02  3.3700e+00]
 [ 6.0000e+00  3.3185e+02  3.3804e+02 -6.1900e+00]
 [ 7.0000e+00  3.7130e+02  3.7125e+02  5.0000e-02]
 [ 8.0000e+00  3.0700e+02  3.1657e+02 -9.5700e+00]
 [ 9.0000e+00  1.8635e+02  1.9125e+02 -4.9000e+00]
 [ 1.0000e+01  3.9409e+02  3.8959e+02  4.5000e+00]
 [ 1.1000e+01  3.0115e+02  3.1498e+02 -1.3830e+01]
 [ 1.2000e+01  3.4665e+02  3.4779e+02 -1.1400e+00]
 [ 1.3000e+01  3.5298e+02  3.6176e+02 -

# 3.Generate Events

## Required input 
- Cif file, the same as local cluster expansion

    path is defined in the prim_cif_name

## Output
- events.json

    store all events in the supercell 

- event\_kernel.csv

    event kernel for coupling between events

- (0th\_reference\_local\_env.cif)

    this is not the input for KMC. This is for checking. Should be the same as the local cluster expansion model.

- (debug.log)

    log file containing debug information. It is recommended to check the distance matrix

In [3]:
mobile_ion_identifier_type = "label"
mobile_ion_specie_1_identifier = "Na1"
mobile_ion_specie_2_identifier = "Na2"
prim_cif_name = "file_before_kmc/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif"
local_env_cutoff_dict = {("Na+", "Na+"): 4, ("Na+", "Si4+"): 4}
from kmcpy.event_generator import generate_events

generate_events(
    prim_cif_name=prim_cif_name,
    local_env_cutoff_dict=local_env_cutoff_dict,
    mobile_ion_identifier_type=mobile_ion_identifier_type,
    mobile_ion_specie_1_identifier=mobile_ion_specie_1_identifier,
    mobile_ion_specie_2_identifier=mobile_ion_specie_2_identifier,
    species_to_be_removed=["O2-", "O", "Zr4+", "Zr"],
    distance_matrix_rtol=0.01,
    distance_matrix_atol=0.01,
    find_nearest_if_fail=False,
    convert_to_primitive_cell=True,
    export_local_env_structure=True,
    supercell_shape=[2, 1, 1],
    event_fname="events.json",
    event_kernal_fname="event_kernal.csv",
    verbosity="INFO",
)

Extracting clusters from primitive cell structure. This primitive cell should be bulk structure, grain boundary model not implemented yet.
primitive cell composition after adding oxidation state and removing uninvolved species: 
Na+8 Si4+6
building mobile_ion_specie_1 index list
please check if these are mobile_ion_specie_1:
[ 0.      0.     11.0905] Na+
[0. 0. 0.] Na+
start finding the neighboring sequence of mobile_ion_specie_1s
total number of mobile_ion_specie_1s:2
1th type of reference local_env structure cif file is created. please check
a new type of local environment is recognized with the species (('Na+', 6), ('Si4+', 6)) 
the distance matrix are 
[[0.         4.72706438 4.72706438 5.1025173  5.1025173  6.95563228
  3.70984167 3.05596884 3.16220005 6.49240171 6.543076   6.19579405]
 [4.72706438 0.         5.1025173  4.72706438 6.95563228 5.1025173
  3.16220005 3.70984167 6.543076   3.05596884 6.19579405 6.49240171]
 [4.72706438 5.1025173  0.         6.95563228 4.72706438 5.102

Saving: events.json
Generating event kernal ...
Saving into: event_kernal.csv


{(('Na+', 6),
  ('Si4+',
   6)): <kmcpy.event_generator.neighbor_info_matcher at 0x7fcdbcc55940>}

# 4.Run KMC simulation
## Required input 
- input.json

    specify the parameter of kmc simulation
- initial_state.json

    initial occupation, typically obtained from a cluster expansion result
## Output

- standard output:
    
    contains information in each step

- \*.csv.gz file:

    contains the information of occupation, displacement, hopping counter, and results, respectively

In [1]:
from kmcpy.io import InputSet
from kmcpy.kmc import KMC
import numpy as np

inputset = InputSet.from_json("file_before_kmc/test_input_v3.json")

print(inputset._parameters.keys())
print(inputset._parameters["mc_results"])
inputset.parameter_checker()

inputset.load_occ(verbose=True)

print("occ", inputset._parameters["occ"])
inputset.set_parameter("use_numpy_random_kernel", False)
kmc = KMC()
events_initialized = kmc.initialization(**inputset._parameters)  # v in 10^13 hz

# # step 2 compute the site kernal (used for kmc run)
kmc.load_site_event_list(inputset._parameters["event_kernel"])

# # step 3 run kmc
kmc_tracker = kmc.run_from_database(events=events_initialized, **inputset._parameters)


print(kmc_tracker.return_current_info())

dict_keys(['v', 'equ_pass', 'kmc_pass', 'supercell_shape', 'fitting_results', 'fitting_results_site', 'lce_fname', 'lce_site_fname', 'prim_fname', 'event_fname', 'event_kernel', 'mc_results', 'T', 'comp', 'structure_idx', 'immutable_sites', 'verbose', 'random_seed', 'use_numpy_random_kernel', 'convert_to_primitive_cell', 'q', 'elem_hop_distance', 'dimension'])
./file_before_kmc/initial_state.json
immutable sites are [0, 1, 2, 3, 4, 5, 6, 7, 32, 33, 34, 35, 36, 37]
selected sites are [0, 1, 2, 3, 4, 5, 6, 7, 32, 33, 34, 35, 36, 37]
converting the occupation raw data to dimension: (42, 2, 1, 1)
occupation_chebyshev: [-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
occ [-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
Initializing kMC calculations with pirm.cif at ./file_before_kmc/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif ...
Supercell Shape:
 [[2 0 0]
 [0 1 0]
 [0 0 1]]
Converting to the supercell ...
removing

/home/jerry/work/kMCpy/kmcpy/tracker.py:188: RuntimeWarning: invalid value encountered in divide
  corr_factor = np.linalg.norm(self.displacement, axis=1) ** 2 / (
